# Etapa 5: **Análisis de las ubicaciones de los sitios de lanzamiento con Folium**


La tasa de éxito del lanzamiento puede depender de muchos factores, como la masa de la carga útil, el tipo de órbita, etc. También puede depender de la ubicación y las proximidades de un lugar de lanzamiento, es decir, de la posición inicial de las trayectorias de los cohetes. Encontrar una ubicación óptima para la construcción de una base de lanzamiento implica ciertamente muchos factores y es de esperar que podamos descubrir algunos de ellos analizando las ubicaciones de las bases de lanzamiento existentes.

En los anteriores laboratorios de análisis de datos exploratorios, has visualizado el conjunto de datos de lanzamiento de SpaceX utilizando `matplotlib` y `seaborn` y has descubierto algunas correlaciones preliminares entre el lugar de lanzamiento y las tasas de éxito. En este laboratorio, realizarás un análisis visual más interactivo utilizando `Folium`.


## Objetivos


Este laboratorio contiene las siguientes tareas:

* **Tarea 1:** Marcar todos los sitios de lanzamiento en un mapa
* **Tarea 2:** Marcar en el mapa los lanzamientos exitosos/fracasados de cada sitio
* **Tarea 3:** Calcular las distancias entre un sitio de lanzamiento y sus proximidades

Después de completar las tareas anteriores, deberías ser capaz de encontrar algunos patrones geográficos sobre los sitios de lanzamiento.


Primero vamos a importar los paquetes de Python necesarios para este laboratorio:


In [63]:
import os
import pandas as pd
try:
    import folium
except ImportError as e:
    !pip install folium wget
    import folium

In [64]:
from folium.plugins import MarkerCluster
from folium.plugins import MousePosition
from folium.features import DivIcon

## Tarea 1: Marcar todos los lugares de lanzamiento en un mapa


En primer lugar, vamos a intentar añadir la ubicación de cada sitio en un mapa utilizando las coordenadas de latitud y longitud del sitio


El siguiente conjunto de datos con el nombre `spacex_launch_geo.csv` es un conjunto de datos aumentado con la latitud y la longitud añadidas para cada sitio.


In [65]:
spacex_csv_file = 'spacex_launch_geo.csv'
spacex_df=pd.read_csv(spacex_csv_file)
spacex_df.head()

,Flight Number,Date,Time (UTC),Booster Version,Launch Site,Payload,Payload Mass (kg),Orbit,Customer,Landing Outcome,class,Lat,Long
0,1,2010-06-04,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0.0,LEO,SpaceX,Failure (parachute),0,28.562302,-80.577356
1,2,2010-12-08,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel o...",0.0,LEO (ISS),NASA (COTS) NRO,Failure (parachute),0,28.562302,-80.577356
2,3,2012-05-22,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2+,525.0,LEO (ISS),NASA (COTS),No attempt,0,28.562302,-80.577356
3,4,2012-10-08,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356
4,5,2013-03-01,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356


Ahora, puedes echar un vistazo a las coordenadas de cada sitio.


In [66]:
# Selecciona las columnas relevantes: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
#Inserta tu código aquí
# for x in launch_sites_df.iterrows():
  # print(fila["Launch Site"], fila["Lat"], fila["Long"], fila["Class"])

In [67]:
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


Las coordenadas anteriores son simples números que no pueden darle ninguna idea intuitiva sobre dónde están esos lugares de lanzamiento. Si eres muy bueno en geografía, puedes interpretar esos números directamente en tu mente. Si no, tampoco pasa nada. Visualicemos esas ubicaciones fijándolas en un mapa.


Primero necesitamos crear un objeto `Map` de folium, con una ubicación central inicial para ser el Centro Espacial Johnson de la NASA en Houston, Texas.

In [68]:
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

Podríamos utilizar `folium.Circle` para añadir un área circular resaltada con una etiqueta de texto en una coordenada específica. Por ejemplo,


In [69]:
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))

marker = folium.map.Marker(
    nasa_coordinate,
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )

site_map.add_child(circle)
site_map.add_child(marker)

y deberías encontrar un pequeño círculo amarillo cerca de la ciudad de Houston y puedes ampliarlo para ver un círculo más grande.


Ahora, agreguemos un círculo para cada sitio de lanzamiento en el marco de datos "sitios de lanzamiento".


### Crear y añadir `folium.Circle` y `folium.Marker` para cada sitio de lanzamiento en el mapa del sitio

Un ejemplo de folium.Circle:


` folium.Circle([coordenadas], color='#d35400', radius=50, fill=True).add_child(folium.Popup((site['Launch Site'])))`


Un ejemplo de folium.Marker:



```
folium.Marker(coordinate,
icon=DivIcon(icon_size=(20,20),
icon_anchor=(0,0),
html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % site['Launch Site'], ))
```


In [70]:
# Inserta tu código aquí
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

for index, site in launch_sites_df.iterrows():
    circle = folium.Circle([site["Lat"], site["Long"]], color='#d35400', radius=50, fill = True).add_child(folium.Popup((site['Launch Site'])))
    marker = folium.Marker([site["Lat"], site["Long"]], icon=DivIcon(icon_size=(20,20), icon_anchor=(0,0),
              html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % site['Launch Site'], ))
    site_map.add_child(circle)
    site_map.add_child(marker)

site_map

El mapa generado con los lugares de lanzamiento marcados debería tener un aspecto similar al siguiente:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_markers.png" />
</center>


Ahora, puedes explorar el mapa acercando o alejando las zonas marcadas
y tratar de responder a las siguientes preguntas:

* ¿Están todas las zonas de lanzamiento cerca de la línea del Ecuador?
* ¿Están todos los puntos de lanzamiento muy cerca de la costa?

Intenta también explicar tus conclusiones.


# Tarea 2: Marcar en el mapa los lanzamientos exitosos/fracasados de cada sitio


A continuación, vamos a intentar mejorar el mapa añadiendo los resultados de los lanzamientos de cada emplazamiento, y ver qué emplazamientos tienen un alto índice de éxito.
Recordemos que el marco de datos spacex_df contiene registros detallados de los lanzamientos, y la columna `clase` indica si el lanzamiento fue exitoso o no


In [71]:
spacex_df.tail(10)

,Flight Number,Date,Time (UTC),Booster Version,Launch Site,Payload,Payload Mass (kg),Orbit,Customer,Landing Outcome,class,Lat,Long
46,43,2017-10-11,22:53:00,F9 FT B1031.2,KSC LC-39A,SES-11 / EchoStar 105,5200.00,GTO,SES EchoStar,Success (drone ship),1,28.573255,-80.646895
47,44,2017-10-30,19:34:00,F9 B4 B1042.1,KSC LC-39A,Koreasat 5A,3500.00,GTO,KT Corporation,Success (drone ship),1,28.573255,-80.646895
48,54,2018-05-11,20:14:00,F9 B5 B1046.1,KSC LC-39A,Bangabandhu-1,3600.00,GTO,Thales-Alenia/BTRC,Success (drone ship),1,28.573255,-80.646895
49,45,2017-12-15,15:36:00,F9 FT B1035.2,CCAFS SLC-40,SpaceX CRS-13,2205.00,LEO (ISS),NASA (CRS),Success (ground pad),1,28.563197,-80.576820
50,47,2018-01-08,1:00:00,F9 B4 B1043.1,CCAFS SLC-40,Zuma,3696.65,LEO,Northrop Grumman,Success (ground pad),1,28.563197,-80.576820
51,48,2018-01-31,21:25:00,F9 FT B1032.2,CCAFS SLC-40,GovSat-1 / SES-16,4230.00,GTO,SES,Controlled (ocean),0,28.563197,-80.576820
52,50,2018-03-06,5:33:00,F9 B4 B1044,CCAFS SLC-40,Hispasat 30W-6 PODSat,6092.00,GTO,Hispasat NovaWurks,No attempt,0,28.563197,-80.576820
53,52,2018-04-02,20:30:00,F9 B4 B1039.2,CCAFS SLC-40,SpaceX CRS-14,2647.00,LEO (ISS),NASA (CRS),No attempt,0,28.563197,-80.576820
54,53,2018-04-18,22:51:00,F9 B4 B1045.1,CCAFS SLC-40,Transiting Exoplanet Survey Satellite (TESS),362.00,HEO,NASA (LSP),Success (drone ship),1,28.563197,-80.576820
55,56,2018-06-04,4:45:00,F9 B4 B1040.2,CCAFS SLC-40,SES-12,5384.00,GTO,SES,No attempt,0,28.563197,-80.576820


A continuación, vamos a crear marcadores para todos los registros de lanzamiento.
Si un lanzamiento fue exitoso `(class=1)`, entonces usamos un marcador verde y si un lanzamiento fue fallido, usamos un marcador rojo `(class=0)`.


Tenga en cuenta que un lanzamiento sólo se produce en uno de los cuatro sitios de lanzamiento, lo que significa que muchos registros de lanzamiento tendrán exactamente la misma coordenada. Los grupos de marcadores pueden ser una buena forma de simplificar un mapa que contenga muchos marcadores con la misma coordenada.


Primero vamos a crear un objeto `MarkerCluster`.


In [72]:
marker_cluster = MarkerCluster()


### Crea una nueva columna en el dataframe `spacex_df` llamada `marker_color` para almacenar los colores de los marcadores basados en el valor `class


Crea una función llamada `assign_marker_color` que devuelva `green` o `red` en función de si `class` (argumento) es 1 o 0. Aplica esta función a la columna `class` dataset mediante el método `df["columna"].apply(funcion)`. Por último, guardala en la columna `marker_color`

In [73]:
# Forma mas simple de hacerlo
"""
colores= []
for clase in spacex_df["class"]:
  if clase == 1:
    colores.append("green")
  else:
    colores.append("red")

print(colores)

spacex_df["marker_color"] = colores
spacex_df
"""

'\ncolores= []\nfor clase in spacex_df["class"]:\n  if clase == 1:\n    colores.append("green")\n  else:\n    colores.append("red")\n\nprint(colores)\n\nspacex_df["marker_color"] = colores\nspacex_df\n'

In [74]:
# Inserta tu código aquí
def assign_marker_color(clase):
    if clase == 1:
        return 'green'
    else:
        return 'red'

spacex_df['marker_color'] = spacex_df['class'].apply(assign_marker_color)

spacex_df.tail(10)

,Flight Number,Date,Time (UTC),Booster Version,Launch Site,Payload,Payload Mass (kg),Orbit,Customer,Landing Outcome,class,Lat,Long,marker_color
46,43,2017-10-11,22:53:00,F9 FT B1031.2,KSC LC-39A,SES-11 / EchoStar 105,5200.00,GTO,SES EchoStar,Success (drone ship),1,28.573255,-80.646895,green
47,44,2017-10-30,19:34:00,F9 B4 B1042.1,KSC LC-39A,Koreasat 5A,3500.00,GTO,KT Corporation,Success (drone ship),1,28.573255,-80.646895,green
48,54,2018-05-11,20:14:00,F9 B5 B1046.1,KSC LC-39A,Bangabandhu-1,3600.00,GTO,Thales-Alenia/BTRC,Success (drone ship),1,28.573255,-80.646895,green
49,45,2017-12-15,15:36:00,F9 FT B1035.2,CCAFS SLC-40,SpaceX CRS-13,2205.00,LEO (ISS),NASA (CRS),Success (ground pad),1,28.563197,-80.576820,green
50,47,2018-01-08,1:00:00,F9 B4 B1043.1,CCAFS SLC-40,Zuma,3696.65,LEO,Northrop Grumman,Success (ground pad),1,28.563197,-80.576820,green
51,48,2018-01-31,21:25:00,F9 FT B1032.2,CCAFS SLC-40,GovSat-1 / SES-16,4230.00,GTO,SES,Controlled (ocean),0,28.563197,-80.576820,red
52,50,2018-03-06,5:33:00,F9 B4 B1044,CCAFS SLC-40,Hispasat 30W-6 PODSat,6092.00,GTO,Hispasat NovaWurks,No attempt,0,28.563197,-80.576820,red
53,52,2018-04-02,20:30:00,F9 B4 B1039.2,CCAFS SLC-40,SpaceX CRS-14,2647.00,LEO (ISS),NASA (CRS),No attempt,0,28.563197,-80.576820,red
54,53,2018-04-18,22:51:00,F9 B4 B1045.1,CCAFS SLC-40,Transiting Exoplanet Survey Satellite (TESS),362.00,HEO,NASA (LSP),Success (drone ship),1,28.563197,-80.576820,green
55,56,2018-06-04,4:45:00,F9 B4 B1040.2,CCAFS SLC-40,SES-12,5384.00,GTO,SES,No attempt,0,28.563197,-80.576820,red


### Para cada resultado de lanzamiento en el marco de datos `spacex_df`, añadir un `folium.Marker` a `marker_cluster`



```
folium.Marker([coordenadas],
icon=folium.Icon(color='white',
icon_color=record['marker_color']))
```


In [75]:
# Inserta tu código aquí
site_map.add_child(marker_cluster)


for index, record in spacex_df.iterrows():
    marker = folium.Marker([record['Lat'], record['Long']],
                  icon=folium.Icon(color='white', icon_color=record['marker_color']))
    marker_cluster.add_child(marker)
site_map

Su mapa actualizado debería parecerse a las siguientes capturas de pantalla:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster.png" />
</center>


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster_zoomed.png" />
</center>


A partir de los marcadores etiquetados con colores en los grupos de marcadores, debería ser capaz de identificar fácilmente qué sitios de lanzamiento tienen tasas de éxito relativamente altas.


# TAREA 3: Calcular las distancias entre un lugar de lanzamiento y sus proximidades


A continuación, tenemos que explorar y analizar las proximidades de los lugares de lanzamiento.


Primero vamos a añadir un MousePosition en el mapa para obtener las coordenadas de un ratón sobre un punto del mapa. Así, mientras exploras el mapa, puedes encontrar fácilmente las coordenadas de cualquier punto de interés (como el ferrocarril)

In [76]:
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

Ahora acerque el zoom a un lugar de lanzamiento y explore su proximidad para ver si puede encontrar fácilmente alguna vía férrea, carretera, costa, etc. Mueve el ratón hacia estos puntos y marca sus coordenadas (mostradas en la parte superior derecha) para conocer la distancia al lugar de lanzamiento.


Puedes calcular la distancia entre dos puntos del mapa a partir de sus valores `Lat` y `Long` utilizando el siguiente método:


In [77]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # Radio de la tierra en km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

Seleccióna un punto en la línea de costa más cercana utilizando MousePosition para averiguar sus coordenadas. Calcula la distancia entre el punto de la línea de costa y el lugar de lanzamiento.


In [78]:
# Para poder resetear el mapa y generar distianta linea de distancia
# site_map = folium.Map()

In [79]:
# Calcula la distancia a la coordenada de la costa más cercana
# e.g.,: Lat: 28.56343  Lon: -80.57678
ubicacion1 =[28.56343, -80.57678]
ubicacion2 = [28.56326, -80.56789]
coordinates = [ubicacion1, ubicacion2]
distance_coastline = calculate_distance(coordinates[0][0], coordinates[0][1], coordinates[1][0], coordinates[1][1])

Para mostrar la linea que une los puntos utilizaremos el siguiente código:

In [80]:
# Crear un objeto `folium.PolyLine` utilizando las coordenadas de la línea de costa y la coordenada del lugar de lanzamiento
lines=folium.PolyLine(locations=coordinates, weight=1)
site_map.add_child(lines)

Después de obtener su coordenada, crea un `folium.Marker` para mostrar la distancia


In [ ]:
# Crear y añadir un folio.Marcador en el punto de costa más cercano seleccionado en el mapa
# Muestra la distancia entre el punto de la línea de costa y el lugar de lanzamiento utilizando la propiedad del icono

distance_marker = folium.Marker(
   coordinate,
   icon=DivIcon(
       icon_size=(20,20),
       icon_anchor=(0,0),
       html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance),
       )
   )

In [82]:
#Inserta tu código aquí
coordinates = [
    [28.56342, -80.57674],
    [30.56367, -80.57163]]

lines=folium.PolyLine(locations=coordinates, weight=1)
site_map.add_child(lines)
distance = calculate_distance(coordinates[0][0], coordinates[0][1], coordinates[1][0], coordinates[1][1])
distance_circle = folium.Marker(
    [28.56342, -80.56794],
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance),
        )
    )
site_map.add_child(distance_circle)
site_map

El mapa actualizado con la línea de distancia debería verse como la siguiente captura de pantalla:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_distance.png" />
</center>


 Del mismo modo, puede dibujar una línea entre un lugar de lanzamiento y su ciudad, ferrocarril, carretera, etc. más cercanos. Para ello, es necesario utilizar `MousePosition` para encontrar sus coordenadas en el mapa


El símbolo de una linea ferrofiaría puede tener este aspecto:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/railway.png" />
</center>


Un símbolo de carretera puede tener este aspecto:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/highway.png" />
</center>


Una ciudad puede tener este aspecto:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/city.png" />
</center>


Crea dos mapas que muestren la distancia hasta una interesección en la autopista y otro hasta una ciudad.

In [ ]:
#Inserta tu código aquí


In [ ]:

#Distancia a una intersección con ferrocarril
coordinates = [
    [28.56342, -80.57674],
    [28.411780, -80.820630]]

lines=folium.PolyLine(locations=coordinates, weight=1)
site_map.add_child(lines)
distance = calculate_distance(coordinates[0][0], coordinates[0][1], coordinates[1][0], coordinates[1][1])
distance_circle = folium.Marker(
    [28.411780, -80.820630],
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#252526;"><b>%s</b></div>' % "{:10.2f} KM".format(distance),
        )
    )
site_map.add_child(distance_circle)
site_map

In [ ]:
#Distancia a la ciudad de Orlando

coordinates = [
    [28.56342, -80.57674],
    [28.5383, -81.3792]]

lines=folium.PolyLine(locations=coordinates, weight=1)
site_map.add_child(lines)
distance = calculate_distance(coordinates[0][0], coordinates[0][1], coordinates[1][0], coordinates[1][1])
distance_circle = folium.Marker(
    [28.5383, -81.3792],
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#252526;"><b>%s</b></div>' % "{:10.2f} KM".format(distance),
        )
    )
site_map.add_child(distance_circle)
site_map

Después de trazar las líneas de distancia a las proximidades, podrá responder fácilmente a las siguientes preguntas:

* ¿Los lugares de lanzamiento están cerca de las vías férreas?
* ¿Están las zonas de lanzamiento cerca de las autopistas?
* ¿Están las zonas de lanzamiento cerca de la costa?
* ¿Los lugares de lanzamiento se mantienen a cierta distancia de las ciudades?

Intenta también explicar sus conclusiones.
